<a href="https://colab.research.google.com/github/cvillad/covid_xray_model/blob/master/download_covid_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import random
import shutil
import pandas as pd

!pip install imutils

from imutils import paths

%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
project_path = '/content/gdrive/Shared drives/Data Mining Project/Kaggle/covid19-xray'
dataset_path = '/content/gdrive/Shared drives/Data Mining Project/dataset'
os.environ['KAGGLE_CONFIG_DIR'] = '/content/gdrive/Shared drives/Data Mining Project/Kaggle'

%cd /content/gdrive/Shared drives/Data Mining Project/Kaggle/covid19-xray

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/Shared drives/Data Mining Project/Kaggle/covid19-xray


In [0]:
%%bash
rm -rf ../../dataset
mkdir -p ../../dataset/covid
mkdir -p ../../dataset/normal
mkdir -p ../../dataset/pneumonia
mkdir -p input/covid-chest-xray
mkdir -p input/chest-xray-pneumonia/chest_xray

In [0]:
# download covid dataset
covid_dataset_path = project_path + '/input/covid-chest-xray'
%cd /content/gdrive/Shared drives/Data Mining Project/Kaggle/covid19-xray/input/covid-chest-xray

!kaggle datasets download -d bachrr/covid-chest-xray
!unzip -qq \*.zip
%rm *.zip

/content/gdrive/Shared drives/Data Mining Project/Kaggle/covid19-xray/input/covid-chest-xray
 99% 209M/211M [00:05<00:00, 93.7MB/s]
100% 211M/211M [00:05<00:00, 40.0MB/s]


In [0]:
# construct the path to the metadata CSV file and load it
csvPath = os.path.sep.join([covid_dataset_path, "metadata.csv"])
df = pd.read_csv(csvPath)

# loop over the rows of the COVID-19 data frame
for (i, row) in df.iterrows():
    # if (1) the current case is not COVID-19 or (2) this is not
    # a 'PA' view, then ignore the row
    if row["view"] != "PA" or row["finding"]!="COVID-19":
        continue

    # build the path to the input image file
    imagePath = os.path.sep.join([covid_dataset_path, "images", row["filename"]])

    # if the input image file does not exist (there are some errors in
    # the COVID-19 metadeta file), ignore the row
    if not os.path.exists(imagePath):
        continue

    filename = row["filename"].split(os.path.sep)[-1]
    outputPath = os.path.sep.join([f"{dataset_path}/covid", filename])

    # extract the filename from the image path and then construct the
    # path to the copied image file

    # copy the image
    shutil.copy2(imagePath, outputPath)

In [0]:
# download pneumonia dataset
pneumonia_dataset_path = project_path + '/input/chest-xray-pneumonia/chest_xray'
%cd /content/gdrive/Shared drives/Data Mining Project/Kaggle/covid19-xray/input/chest-xray-pneumonia

!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip -qq \*.zip
%rm *.zip

/content/gdrive/Shared drives/Data Mining Project/Kaggle/covid19-xray/input/chest-xray-pneumonia
100% 2.29G/2.29G [00:32<00:00, 31.9MB/s]
100% 2.29G/2.29G [00:32<00:00, 76.3MB/s]



In [0]:
# build normal xray dataset
samples = 150
random_seed=40
basePath = os.path.sep.join([pneumonia_dataset_path, "train", "NORMAL"])
imagePaths = list(paths.list_images(basePath))

# randomly sample the image paths
random.seed(random_seed)
random.shuffle(imagePaths)
imagePaths = imagePaths[:samples]

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    # extract the filename from the image path and then construct the
    # path to the copied image file
    filename = imagePath.split(os.path.sep)[-1]
    outputPath = os.path.sep.join([f"{dataset_path}/normal", filename])

    # copy the image
    shutil.copy2(imagePath, outputPath)

In [0]:
# build pneumonia xray dataset
samples = 150
basePath = os.path.sep.join([pneumonia_dataset_path, "train", "PNEUMONIA"])
imagePaths = list(paths.list_images(basePath))

# randomly sample the image paths
random.seed(random_seed)
random.shuffle(imagePaths)
imagePaths = imagePaths[:samples]

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    # extract the filename from the image path and then construct the
    # path to the copied image file
    filename = imagePath.split(os.path.sep)[-1]
    outputPath = os.path.sep.join([f"{dataset_path}/pneumonia", filename])

    # copy the image
    shutil.copy2(imagePath, outputPath)